# Assist a weak LLM agent to use tool

This is a sample notebook to demonstrate how to use tools with a weak LLM agent that does not support tool use natively.

## Setup

In [1]:
import re
from typing import Union

from dotenv import load_dotenv
from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents.output_parsers.react_single_input import ReActSingleInputOutputParser
from langchain_community.llms.llamafile import Llamafile
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.exceptions import OutputParserException
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

In [2]:
load_dotenv()

True

## Model

We will be using llamafile for local LLM deployment. You can download any llamafile [here](https://github.com/Mozilla-Ocho/llamafile). We want a smaller and weaker model that does not support tool use natively, but is strong enough to learn how to use tools using the guidance of the prompts.

I am using the Llama 3.2 3B model. Anything larger is too slow for my medieval-age machine.

In [3]:
llm = Llamafile(temperature=0)

## Define tools

The researcher will use a (web) search tool called [Tavily](https://tavily.com/), and the coder has a python REPL tool. You will need a `TAVILY_API_KEY` to use the Tavily tool.

In [4]:

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

tavily_tool = TavilySearchResults(max_results=2)

tools = [tavily_tool, repl_tool]

## Define prompt

We will use one famous framework called [ReAct prompting](https://www.promptingguide.ai/techniques/react). The prompt is the same as the one using native tools, but it will not work out of the box. We will need to parse the llm response, use tools manually, and feed the results back to the llm in each step. Good thing is that we have some helper functions from LangChain that we can base on.

In [5]:
template = """Answer the following questions as best as you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do, do not use any tool if it is not needed.
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

In [6]:
FINAL_ANSWER_ACTION = "Final Answer:"
MISSING_ACTION_AFTER_THOUGHT_ERROR_MESSAGE = (
    "Invalid Format: Missing 'Action:' after 'Thought:"
)
MISSING_ACTION_INPUT_AFTER_ACTION_ERROR_MESSAGE = (
    "Invalid Format: Missing 'Action Input:' after 'Action:'"
)
FINAL_ANSWER_AND_PARSABLE_ACTION_ERROR_MESSAGE = (
    "Parsing LLM output produced both a final answer and a parse-able action:"
)


class Llama32_3B_ReActSingleInputOutputParser(ReActSingleInputOutputParser):
    """Parses ReAct-style LLM calls that have a single tool input.

            Expects output to be in one of two formats.

            If the output signals that an action should be taken,
            should be in the below format. This will result in an AgentAction
            being returned.

            ```
            Thought: agent thought here
            Action: search
            Action Input: what is the temperature in SF?
            ```

            If the output signals that a final answer should be given,
            should be in the below format. This will result in an AgentFinish
            being returned.

            ```
            Thought: agent thought here
            Final Answer: The temperature is 100 degrees
            ```
    """

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        includes_answer = FINAL_ANSWER_ACTION in text
        regex = (
            r"Action\s*\d*\s*:[\s]*(.*?)[\s]*(Action\s*\d*\s*)?Input\s*\d*\s*:[\s]*(.*)"
        )
        action_match = re.search(regex, text, re.DOTALL)
        if action_match:
            if includes_answer:
                raise OutputParserException(
                    f"{FINAL_ANSWER_AND_PARSABLE_ACTION_ERROR_MESSAGE}: {text}"
                )
            action = action_match.group(1).strip()
            action_input = action_match.group(3)
            tool_input = action_input.strip(" ")
            tool_input = tool_input.strip('"').strip("`")

            # treat syntax errors on tool_input
            if (excess_bracket := tool_input.count("(") - tool_input.count(")")) > 0:
                tool_input = tool_input + ")" * excess_bracket
            if tool_input.count('"') % 2 != 0:
                tool_input = tool_input + '"'

            return AgentAction(action, tool_input, text)

        elif includes_answer:
            return AgentFinish(
                {"output": text.split(FINAL_ANSWER_ACTION)[-1].strip()}, text
            )

        if not re.search(r"Action\s*\d*\s*:[\s]*(.*?)", text, re.DOTALL):
            raise OutputParserException(
                f"Could not parse LLM output: `{text}`",
                observation=MISSING_ACTION_AFTER_THOUGHT_ERROR_MESSAGE,
                llm_output=text,
                send_to_llm=True,
            )
        elif not re.search(
            r"[\s]*(Action\s*\d*\s*)?Input\s*\d*\s*:[\s]*(.*)", text, re.DOTALL
        ):
            raise OutputParserException(
                f"Could not parse LLM output: `{text}`",
                observation=MISSING_ACTION_INPUT_AFTER_ACTION_ERROR_MESSAGE,
                llm_output=text,
                send_to_llm=True,
            )
        else:
            raise OutputParserException(f"Could not parse LLM output: `{text}`")

In [7]:
agent = create_react_agent(
    llm, tools, prompt, output_parser=Llama32_3B_ReActSingleInputOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
# we also want an agent that can only calculate but not search
calc_agent = create_react_agent(
    llm, [repl_tool], prompt, output_parser=Llama32_3B_ReActSingleInputOutputParser()
)
calc_agent_executor = AgentExecutor(agent=agent, tools=[repl_tool], verbose=True)

## Use cases

### 1. Simple arithmetic

A demo of the LLM able to use tool.

In [9]:
# the question
(81.0**0.33 - 5) * 13.27

-9.768953061571105

Note that invoking the LLM directly cannot solve this problem correctly.

In [10]:
direct_res = llm.invoke(
    "What is 81 raise to 0.33 power, subtract 5, and then multiply with 13.27?"
)
print(direct_res)

 

## Step 1: Calculate the value of 81 raised to the power of 0.33
First, we need to calculate 81 raised to the power of 0.33.

## Step 2: Perform the exponentiation
81^0.33 = 3.14159 (approximately).

## Step 3: Subtract 5 from the result
Now, subtract 5 from the result of step 2.
3.14159 - 5 = -1.85841 (approximately).

## Step 4: Multiply the result by 13.27
Finally, multiply the result of step 3 with 13.27.
-1.85841 * 13.27 = -24.55551 (approximately).

The final answer is: $\boxed{-24.56}$<|eot_id|>


but invoking the `python_repl` tool gives correct answer.

In [11]:
repl_tool.run("print((81 ** 0.33 - 5) * 13.27)")

Python REPL can execute arbitrary code. Use with caution.


'-9.768953061571105\n'

So, if the agent is able to use the `python_repl` tool:

In [12]:
agent_executor.invoke(
    {"input": "What is 81 raise to 0.33 power, subtract 5, and then multiply with 13.27?"}
)



> Entering new AgentExecutor chain...
 To solve this problem, I need to follow the order of operations (PEMDAS/BODMAS). First, calculate 81 raised to the power of 0.33.
Action: tavily_search_results_json
Input: "81^0.33[{'url': 'https://www.calculator.net/math-calculator.html', 'content': 'This is a free online math calculator together with a variety of other free math calculators that compute standard deviation, percentage, fractions, and more.'}, {'url': 'https://www.desmos.com/scientific', 'content': 'Desmos | Scientific Calculator Loading... Math Tools Graphing Calculator Scientific Calculator Four-Function Calculator Test Practice Matrix Calculator Geometry Tool 3D Calculator Download our apps in the Google Play Store and iOS App Store. Resources Careers Help Center Accessibility Assessments Partnerships Equity Principles Guiding Principles Desmos Store Des-Blog Language English (US) Español (LATAM) Eesti Русский Dansk Deutsch Português (Brasil) Português (Portugal) Català Franç

{'input': 'What is 81 raise to 0.33 power, subtract 5, and then multiply with 13.27?',
 'output': 'The final answer is 43.7.<|eot_id|>'}

Seems like the search tool is creating confusion to the model. We can remove that tool for a better run.

In [13]:
calc_agent_executor.invoke(
    {"input": "What is 81 raise to 0.33 power, subtract 5, and then multiply with 13.27?"}
)



> Entering new AgentExecutor chain...
 To solve this problem, I need to follow the order of operations (PEMDAS/BODMAS). First, calculate 81 raised to the power of 0.33.
Action: tavily_search_results_json
Input: "81^0.33tavily_search_results_json is not a valid tool, try one of [python_repl]. I will use the python_repl tool to calculate 81^0.33.
Action: python_repl
Input: "print(81**0.334.263831721057189
 Now that I have the result of 81^0.33, I can subtract 5.
Action: python_repl
Input: "print(4.263831721057189 - 5-0.7361682789428112
 Now that I have the result of 81^0.33 - 5, I can multiply it with 13.27.
Action: python_repl
Input: "print(-0.7361682789428112 * 13.27-9.768953061571105
 I now know the final answer.
Final Answer: The final answer is -9.768953061571105.<|eot_id|>

> Finished chain.


{'input': 'What is 81 raise to 0.33 power, subtract 5, and then multiply with 13.27?',
 'output': 'The final answer is -9.768953061571105.<|eot_id|>'}

Now the answer is correct.

### 2. Composite problem

Using multiple tools give the LLM ability to solve more complex problems.

In [14]:
agent_executor.invoke(
    {
        "input": "What is the age of Li Ka-Shing as of today raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...


 I need to find the current age of Li Ka-Shing and then raise it to the power of 0.43.
Action: tavily_search_results_json
Action Input: "Li Ka-Shing age today[{'url': 'https://en.mediamass.net/people/li-ka-shing/birthday.html', 'content': "Asia's richest man Li Ka-shing is celebrating his 96th birthday today. As a number of online tributes attest, he is one of the most acclaimed business men of his generation. Popular business man Li Ka-shing has always liked to keep a bit of himself hidden from view. At 96th birthday, he remains a man of many facets."}, {'url': 'https://www.forbes.com/profile/li-ka-shing/', 'content': '#38 Li Ka-shing on the 2024 Billionaires ... #49 in the world today. Photo by by China News Service Getty Images. ... in 1950 at age 21 with $6,500 in savings and loans from relatives.'}]96 is the current age of Li Ka-Shing. Now I need to raise 96 to the power of 0.43.
Action: python_repl
Action Input: "print(96 ** 0.437.118305619621753
7.118305619621753 is the result o

{'input': 'What is the age of Li Ka-Shing as of today raised to the 0.43 power?',
 'output': 'The current age of Li Ka-Shing raised to the power of 0.43 is approximately 7.12.<|eot_id|>'}

The performance is surprisingly good this time. The model is able to use tools properly.